The Goal is to merge `merged_insurance_data.csv`, `ACS_S1901_YearZIP.csv`, `ACS_Demographics_YearZIP.csv`, and `HPI_ZIP_year_affected.csv`

In [13]:
import pandas as pd

We want our final table to contain the columns:
1. Zip Code
2. Year
3. HPI
4. HPI Annual Change (%)
5. Insurance Coverage (%) = Active Insurance/Total Population (from transposed ACS_Demographics)
6. Total Population
7. (%) Age Brackets
8. (%) White v Non-White
9. (%) Household Income Brackets


In [14]:
# Read the CSV files
hpi_data = pd.read_csv('data/HPI_ZIP_year_affected.csv')
insurance_data = pd.read_csv('data/merged_insurance_data.csv')
acs_income_data = pd.read_csv('data/ACS_S1901_YearZIP.csv')
acs_demographics_data = pd.read_csv('data/ACS_Demographics_YearZIP.csv')

In [15]:
# Pivot the income dataset to create columns for each income label
income_pivoted = acs_income_data.pivot_table(
    index=['Zip Code', 'Year'],
    columns='Label',
    values='Estimate',
    aggfunc='first'
).reset_index()

# Rename columns to reflect their meanings clearly
income_pivoted.rename(
    columns={
        'Total (Households)': 'Total (Households)',
        'Less than $10,000': 'Less than $10,000',
        '$10,000 to $14,999': '$10,000 to $14,999',
        '$15,000 to $24,999': '$15,000 to $24,999',
        '$25,000 to $34,999': '$25,000 to $34,999',
        '$35,000 to $49,999': '$35,000 to $49,999',
        '$50,000 to $74,999': '$50,000 to $74,999',
        '$75,000 to $99,999': '$75,000 to $99,999',
        '$100,000 to $149,999': '$100,000 to $149,999',
        '$150,000 to $199,999': '$150,000 to $199,999',
        '$200,000 or more': '$200,000 or more',
        'Median income (dollars)': 'Median income (dollars)',
        'Mean income (dollars)': 'Mean income (dollars)'
    },
    inplace=True
)

desired_order = [
    'Zip Code', 'Year', 'Total (Households)', 'Less than $10,000',
    '$10,000 to $14,999', '$15,000 to $24,999', '$25,000 to $34,999',
    '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999',
    '$100,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
    'Median income (dollars)', 'Mean income (dollars)'
]

# Reorder the DataFrame columns
income_pivoted = income_pivoted[desired_order]

# Display the resulting ACS Income table to the user
print(income_pivoted)

Label  Zip Code  Year  Total (Households)  Less than $10,000  \
0         94503  2017              5505.0                2.3   
1         94503  2018              5442.0                2.3   
2         94503  2019              5303.0                2.4   
3         94503  2020              5147.0                3.8   
4         94503  2021              5622.0                3.5   
..          ...   ...                 ...                ...   
437       95987  2018              1803.0                2.3   
438       95987  2019              1766.0                2.7   
439       95987  2020              1738.0                3.8   
440       95987  2021              1871.0                3.8   
441       95987  2022              1878.0                2.7   

Label  $10,000 to $14,999  $15,000 to $24,999  $25,000 to $34,999  \
0                     3.1                 6.8                 5.0   
1                     4.3                 4.4                 5.0   
2                     4.

In [16]:
demographics_order = [
    'Total population',
    'Male',
    'Female',
    'Under 5 years',
    '5 to 9 years',
    '10 to 14 years',
    '15 to 19 years',
    '20 to 24 years',
    '25 to 34 years',
    '35 to 44 years',
    '45 to 54 years',
    '55 to 59 years',
    '60 to 64 years',
    '65 to 74 years',
    '75 to 84 years',
    '85 years and over',
    'Median age (years)'
]

# Pivot the demographics data to directly use the Percent values
demographics_pivoted_direct = acs_demographics_data.pivot_table(
    index=['Zip Code', 'Year'],
    columns='Label',
    values='Percent',
    aggfunc='first'
).reset_index()

# Reorder columns based on the specified order
existing_columns_direct = ['Zip Code', 'Year'] + [
    col for col in demographics_order if col in demographics_pivoted_direct.columns
]
demographics_pivoted_direct = demographics_pivoted_direct[existing_columns_direct]

print(demographics_pivoted_direct)

Label  Zip Code  Year  Total population  Male  Female  Under 5 years  \
0         94503  2017           20371.0  49.6    50.4            6.6   
1         94503  2018           20306.0  49.7    50.3            6.4   
2         94503  2019           20276.0  49.8    50.2            5.7   
3         94503  2020           20295.0  48.9    51.1            5.7   
4         94503  2021           21874.0  49.4    50.6            4.8   
..          ...   ...               ...   ...     ...            ...   
437       95987  2018            6119.0  50.3    49.7            6.6   
438       95987  2019            5950.0  48.4    51.6            8.4   
439       95987  2020            6040.0  49.3    50.7            9.9   
440       95987  2021            6370.0  51.5    48.5            9.1   
441       95987  2022            6504.0  49.5    50.5            9.1   

Label  5 to 9 years  10 to 14 years  15 to 19 years  20 to 24 years  \
0               6.8             8.3             7.2             

In [17]:
#to csv
income_pivoted.to_csv('data/ACS_Income_Pivoted.csv', index=False)
demographics_pivoted_direct.to_csv('data/ACS_Demographics_Pivoted.csv', index=False)

In [20]:
# Merge the dataframes on 'Zip Code' and 'Year'
merged_data = hpi_data.merge(insurance_data, left_on=['Zip Code', 'Year'], right_on=['ZIP Code', 'Year'], how='left')
merged_data = merged_data.merge(demographics_pivoted_direct, on=['Zip Code', 'Year'], how='left')
merged_data = merged_data.merge(income_pivoted, on=['Zip Code', 'Year'], how='left')

# Select the required columns
final_columns = [
    'Zip Code', 'Year', 'HPI with 2000 base', 'Annual Change (%)', 'Active', 'Renewed', 'Total population', 'Male', 'Female', 'Under 5 years',
    '5 to 9 years', '10 to 14 years', '15 to 19 years', '20 to 24 years', '25 to 34 years', '35 to 44 years',
    '45 to 54 years', '55 to 59 years', '60 to 64 years', '65 to 74 years', '75 to 84 years', '85 years and over',
    'Total (Households)', 'Less than $10,000', '$10,000 to $14,999', '$15,000 to $24,999',
    '$25,000 to $34,999', '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999', '$100,000 to $149,999',
    '$150,000 to $199,999', '$200,000 or more', 'Median income (dollars)', 'Mean income (dollars)'
]

final_data = merged_data[final_columns]

# Save the final dataframe to a CSV file
final_data.to_csv('data/final_merged_data.csv', index=False)